In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import scipy.stats as stats
from linearmodels.iv import IV2SLS
import math

## Game Fun: Customer Acquisition through Digital Advertising

**1. Before evaluating the effect of an experiment, it is important to make sure that the experiment was executed correctly. Check whether the test and control groups are probabilistically equivalent on their observables?**

        a. More specific, compare the averages of the income, gender and gamer variables in the test and control groups. You should also report the % difference in the averages. Compute its statistical significance.

In [2]:
gamefun = pd.read_excel('GameFun.xlsx')
gamefun.head()

,id,test,purchase,site,impressions,income,gender,gamer
0,1956,0,0,site1,0,100,1,0
1,45821,1,0,site1,20,70,1,0
2,59690,1,0,site1,22,100,1,0
3,18851,0,0,site1,13,90,1,0
4,60647,1,0,site1,12,60,1,0


In [3]:
comparison = gamefun.groupby('test')['income', 'gender', 'gamer'].agg('mean')
comparison.loc['% diff'] = ((comparison.loc[1] - comparison.loc[0]) / comparison.loc[0]) * 100
comparison

/Users/vivianwu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,income,gender,gamer
test,,,
0,55.166012,0.647905,0.601823
1,54.938236,0.647289,0.601331
% diff,-0.412890,-0.095049,-0.081720


In [4]:
def ttest(df, group, target, standard):
    control = df[df[group] == 0][target]
    test = df[df[group] == 1][target]
    ttest = stats.ttest_ind(control, test)[1]
    result = pd.DataFrame({'variable':[target], 'p-value': [ttest]})
    result['test result'] = np.where(result['p-value'] < standard, 'Reject H0. Control and test groups are not similar',
                                    'Fail to reject H0. Control and test groups are similar.')
    return result

In [5]:
result1 = ttest(gamefun, 'test', 'income', 0.05)
result2 = ttest(gamefun, 'test', 'gender', 0.05)
result3 = ttest(gamefun, 'test', 'gamer', 0.05)

final_result = pd.concat([result1, result2, result3]).reset_index()
final_result

,index,variable,p-value,test result
0,0,income,0.128358,Fail to reject H0. Control and test groups are similar.
1,0,gender,0.906033,Fail to reject H0. Control and test groups are similar.
2,0,gamer,0.926704,Fail to reject H0. Control and test groups are similar.


**2. Evaluate the average purchase rates in the test and control for the following groups. For each comparison, report the average purchase rate for the test, average purchase rate for the control and the absolute difference (not the % difference) between the test and control.**

    a. Comparison 1: All customers

In [28]:
def diff_calculation(df, group, target, variable = None): 
    control = df[df[group] == 0]
    test = df[df[group] == 1]
    
    if variable is not None:
        
        control_mean = control.groupby(variable)[target].agg('mean')
        test_mean = test.groupby(variable)[target].agg('mean')
       
        if len(variable) == 1:
            diff = round(((test_mean[1] - control_mean[1]) - (test_mean[0] - control_mean[0])) * 100, 4)
            print('The average purchase rate for test:', round(test_mean[1] * 100, 4), '/ ', round(test_mean[0] * 100, 4))
            print('The average purchase rate for control:', round(control_mean[1] * 100, 4), '/ ', round(control_mean[0] * 100, 4))
            print('The absolute difference:', abs(diff))
        
        else:
            diff = (test_mean[0, 1] - control_mean[0, 1]) - (test_mean[1, 1] - control_mean[1, 1])
            diff = round(diff * 100, 4)
            print('The average purchase rate for test:', round(test_mean[0, 1] * 100, 4), '/ ', round(test_mean[1, 1] * 100, 4))
            print('The average purchase rate for control:', round(control_mean[0, 1] * 100, 4), '/ ', round(control_mean[1, 1] * 100, 4))
            print('The absolute difference:', abs(diff))        
    
    else:
        diff = round((test[target].agg('mean') - control[target].agg('mean')) * 100, 4)
        print('The average purchase rate for test:', round((test[target].agg('mean') * 100), 4))      
        print('The average purchase rate for control:', round((control[target].agg('mean') * 100), 4))
        print('The absolute difference:', abs(diff))

In [29]:
diff_calculation(gamefun, 'test', 'purchase')

The average purchase rate for test: 7.6822
The average purchase rate for control: 3.6213
The absolute difference: 4.0609


    b. Comparison 2: Male vs Female customers

In [30]:
diff_calculation(gamefun, 'test', 'purchase', ['gender'])

The average purchase rate for test: 7.4575 /  8.0945
The average purchase rate for control: 3.7176 /  3.4442
The absolute difference: 0.9103


    c. Comparison 3: Gamers vs Non-Gamers Customer

In [31]:
diff_calculation(gamefun, 'test', 'purchase', ['gamer'])

The average purchase rate for test: 10.4487 /  3.5092
The average purchase rate for control: 3.5436 /  3.7387
The absolute difference: 7.1346


    d. Comparison 4: Female Gamers vs Male Gamers

In [10]:
diff_calculation(gamefun, 'test', 'purchase', ['gender', 'gamer'])

The average purchase rate for test: 11.0092 /  10.1404
The average purchase rate for control: 3.2041 /  3.7275
The absolute difference: 1.3922


**3. Assess the expected revenue in the test vs. control for the following comparisons:**

    a. Comparison 1: All customers   

In [11]:
control = gamefun[gamefun['test'] == 0]
test = gamefun[gamefun['test'] == 1]

avg_revenue = 37.5

control_purchase = control['purchase'].agg('mean')
test_purchase = test['purchase'].agg('mean')

print('The expected revenue in the control group: $', round(avg_revenue * control_purchase, 2))
print('The expected revenue in the test group: $', round((avg_revenue * test_purchase) , 2))
print('The difference of expected revenue: $', round((avg_revenue * test_purchase - avg_revenue * control_purchase), 2))

The expected revenue in the control group: $ 1.36
The expected revenue in the test group: $ 2.88
The difference of expected revenue: $ 1.52


    b. Comparison 4: Female Gamers vs Male Gamers

In [12]:
control = gamefun[(gamefun['test'] == 0) & (gamefun['gamer'] == 1)]
test = gamefun[(gamefun['test'] == 1) & (gamefun['gamer'] == 1)]

avg_revenue = 37.5

female_gamers_control = control[control['gender'] == 0]['purchase'].mean()
female_gamers_test = test[test['gender'] == 0]['purchase'].mean()
male_gamers_control = control[control['gender'] == 1]['purchase'].mean()
male_gamers_test = test[test['gender'] == 1]['purchase'].mean()

print('The expected revenue in the female gamers control group: $', round(avg_revenue * female_gamers_control, 2))
print('The expected revenue in the female gamers test group: $', round((avg_revenue * female_gamers_test) , 2))

print('The expected revenue in the male gamers control group: $', round(avg_revenue * male_gamers_control, 2))
print('The expected revenue in the male gamers test group: $', round((avg_revenue * male_gamers_test) , 2))

print('---------------------------------------------------------------')
print('The difference of expected revenue for female gamers: $', 
      round((avg_revenue * female_gamers_test - avg_revenue * female_gamers_control), 2))
print('The difference of expected revenue for male gamers: $', 
      round((avg_revenue * male_gamers_test - avg_revenue * male_gamers_control), 2))

The expected revenue in the female gamers control group: $ 1.2
The expected revenue in the female gamers test group: $ 4.13
The expected revenue in the male gamers control group: $ 1.4
The expected revenue in the male gamers test group: $ 3.8
---------------------------------------------------------------
The difference of expected revenue for female gamers: $ 2.93
The difference of expected revenue for male gamers: $ 2.4


## Non-Compliance in Randomized Experiments

**1. The first data scientist advised that one should compare the survival rate of babies whose mothers were offered Vitamin A shots to the survival rate of babies whose mothers were not offered a Vitamin A shot.**

    a. What percent of babies whose mothers were offered Vitamin A shots for their babies died? 

In [13]:
sommerdeger = pd.read_csv('sommer_deger.csv')
sommerdeger.head()

,instrument,treatment,outcome
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [14]:
sommerdeger.shape

(23682, 3)

In [15]:
groupA = sommerdeger[sommerdeger['instrument'] == 1]
groupB = sommerdeger[sommerdeger['instrument'] == 0]

groupA_mortality = round(groupA['outcome'].agg('mean'), 4) * 100
print('The percent is:', groupA_mortality)

The percent is: 0.38


    b. What percent of babies whose mothers were not offered Vitamin A shots for their babies died?

In [16]:
groupB_mortality = round(groupB['outcome'].agg('mean'), 4) * 100
print('The percent is:', groupB_mortality)

The percent is: 0.64


    c. What is the difference in mortality, and under what assumptions is the difference between these two percentages a valid estimate of the causal impact of receiving vitamin A shots on survival?

In [17]:
diff_mortality = groupA_mortality - groupB_mortality
print('The difference in mortality is:', diff_mortality)

The difference in mortality is: -0.26


Assuming that the only difference between these two groups is whether or not babies' mothers were offered Vitamin A shots for their babies and that there are no other confounding variables that would affect the outcome, the difference in mortality is a valid estimate of the causal impact of receiving Vitamin A shots on survival.

**2. The second data scientist advised that one should compare the survival rates of babies who received Vitamin A shots to babies who did not receive Vitamin A shots.**

    a. What percent of babies who received Vitamin A shots died? 

In [18]:
groupC = sommerdeger[sommerdeger['treatment'] == 1]
groupD = sommerdeger[sommerdeger['treatment'] == 0]

groupC_mortality = round(groupC['outcome'].agg('mean'), 4) * 100
print('The percent is:', groupC_mortality)

The percent is: 0.12


    b. What percent of babies who did not receive Vitamin A shots died?

In [19]:
groupD_mortality = round(groupD['outcome'].agg('mean'), 4) * 100
print('The percent is:', groupD_mortality)

The percent is: 0.77


    c. What is the difference in mortality, and under what assumptions is the difference between these two percentages a valid estimate of the causal impact of receiving vitamin A shots on survival?

In [20]:
diff_mortality = groupC_mortality - groupD_mortality
print('The difference in mortality is:', diff_mortality)

The difference in mortality is: -0.65


Assuming that the only difference between these two groups is whether or not the baby got the vitamin A shot and that there are no other confounding variables that would affect the outcome, the difference in mortality is a valid estimate of the causal impact of receiving Vitamin A shots on survival.

**3. The third data scientist advised that one should consider only babies whose mothers were offered Vitamin A shots, and compare babies who received shots to babies who did not receive shots.**

    a. What percent of babies who received Vitamin A shots died?

In [21]:
groupE = groupA[groupA['treatment'] == 1]
groupF = groupA[groupA['treatment'] == 0]

groupE_mortality = round(groupE['outcome'].agg('mean'), 4) * 100
print('The percent is:', groupE_mortality)

The percent is: 0.12


    b. What percent of babies whose mothers were offered Vitamin A shots, but the mothers did not accept them, died?

In [22]:
groupF_mortality = round(groupF['outcome'].agg('mean'), 4) * 100
print('The percent is:', groupF_mortality)

The percent is: 1.41


    c. What is the difference in mortality, and under what assumptions is the difference between these two percentages a valid estimate of the causal impact of receiving vitamin A shots on survival?

In [23]:
diff_mortality = groupE_mortality - groupF_mortality
print('The difference in mortality is:', diff_mortality)

The difference in mortality is: -1.29


Assuming that the only difference between these two groups (both groups were offered the vitamin A shot) is whether or not the baby got the vitamin A shot and that there are no other confounding variables that would affect the outcome, the difference in mortality is a valid estimate of the causal impact of receiving Vitamin A shots on survival. However, as the mortality rate among babies whose mothers were offered vitamin A shots but not accept them is much higher that the mortality rate among babies who did not receive the shots, it suggests that there may be other factors, such as the health status of babies or their mothers' decision-making process, that influence the outcome.

**4. The fourth data scientist suggested the following Wald estimator for the effect of Vitamin A shots on mortality:**


$$\frac{\% \text{ of babies offered shot that died} - \% \text{ of babies not offered shots that died}}
{ \% \text{ of babies who were offered a shot and received it}}$$


    a. Compute the above Wald estimate for the given dataset.

In [24]:
# calculate two-stage least square
model = IV2SLS.from_formula('outcome ~ 1 + [treatment ~ instrument]', data = sommerdeger)

results = model.fit()
print(results.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                outcome   R-squared:                      0.0015
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0015
No. Observations:               23682   F-statistic:                    7.7551
Date:                Fri, Apr 14 2023   P-value (F-stat)                0.0054
Time:                        12:55:46   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0064     0.0007     8.6299     0.0000      0.0049      0.0078
treatment     -0.0032     0.0012    -2.7848     0.00

In [25]:
print('The Wald estimate is:', round(results.params[1] * 100, 2))

The Wald estimate is: -0.32


    b. What is the standard error for the intent-to-treat estimate recommended by the first data scientist? What is the standard error for the Wald estimate recommended by the fourth data scientist?

In [26]:
# calculate the standard error for the intent-to-treat estimate
se1 = math.sqrt((groupA_mortality/ 100 * (1 - groupA_mortality/ 100)) / len(groupA) +
                (groupB_mortality/ 100 * (1 - groupB_mortality/ 100)) / len(groupB))

print('The standard error for the estimate recommended by the first data scientist:', round(se1, 4))

The standard error for the estimate recommended by the first data scientist: 0.0009


In [27]:
# calculate the standard error for the Wald estimate
print('The standard error for the Wald estimate recommended by the fourth data scientist:', round(results.std_errors[1], 4))

The standard error for the Wald estimate recommended by the fourth data scientist: 0.0012
